### Installing Dependencies

**Step 1: Install Dependencies**
We need to install following components to run pyspark seamlessly:
OpenJDK 8,
Spark Environment,
FindSpark package

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark

**Step 2: Add environment variables**
After installing dependencies, we need to some variables to the environment so that pyspark knows where to look for using dependencies. We can do that using following commands:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.0-bin-hadoop3.2"

**Step 3: Initilize pyspark**
Finally, we just need to initilize pyspark which can be easily achieved using third-party package named findspark as shown below:

In [ ]:
import findspark
findspark.init()

You can try running following commands to check if pyspark is properly installed or not:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Mounting Google Drive

In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/cloud_computing/Project

/content/drive/MyDrive/cloud_computing/Project


# Cleaning dataframes

#### Reading CSV file

In [ ]:
import pandas as pd

# read in CSV file with low_memory=False
df = pd.read_csv('/content/drive/MyDrive/cloud_computing/Project/google_tweets_2022.csv', engine='python')

In [ ]:
df.shape

(792012, 8)

In [ ]:
df.head()

,date,rawContent,replyCount,retweetCount,likeCount,quoteCount,hashtags,cashtags
0,2022-12-30 23:59:54+00:00,@joshnamesnotjo1 thank you google https://t.co...,2.0,1.0,97.0,0,NaN,NaN
1,2022-12-30 23:57:05+00:00,"""wow, librarians still exist? I thought that w...",0.0,1.0,6.0,0,NaN,NaN
2,2022-12-30 23:56:14+00:00,i miss being a kid and slamming out words for ...,1.0,1.0,11.0,0,NaN,NaN
3,2022-12-30 23:52:57+00:00,@realsebastiani I post about republicans and d...,3.0,0.0,7.0,0,NaN,NaN
4,2022-12-30 23:51:47+00:00,@BLKAnonymous @PrettyMill1 @WendellPierce Goog...,1.0,0.0,28.0,0,NaN,NaN


In [ ]:
# Checking for null values
df.isna().sum()

date                 0
rawContent           1
replyCount           6
retweetCount         6
likeCount            6
quoteCount          10
hashtags        669665
cashtags        783921
dtype: int64

#### Dropping null values

In [ ]:
df.dropna(subset=['date','rawContent'],inplace = True)

In [ ]:
df.shape

(792011, 8)

#### Dropping columns

In [ ]:
df.drop(['hashtags', 'cashtags'], axis=1, inplace=True)

In [ ]:
df.columns

Index(['date', 'rawContent', 'replyCount', 'retweetCount', 'likeCount',
       'quoteCount'],
      dtype='object')

#### Filling null values

In [ ]:
df[['replyCount','retweetCount','likeCount','quoteCount']] = df[['replyCount','retweetCount','likeCount','quoteCount']].fillna(0)

In [ ]:
df.isna().sum()

date            0
rawContent      0
replyCount      0
retweetCount    0
likeCount       0
quoteCount      0
dtype: int64

In [ ]:
df.head()

,date,rawContent,replyCount,retweetCount,likeCount,quoteCount
0,2022-12-30 23:59:54+00:00,@joshnamesnotjo1 thank you google https://t.co...,2.0,1.0,97.0,0
1,2022-12-30 23:57:05+00:00,"""wow, librarians still exist? I thought that w...",0.0,1.0,6.0,0
2,2022-12-30 23:56:14+00:00,i miss being a kid and slamming out words for ...,1.0,1.0,11.0,0
3,2022-12-30 23:52:57+00:00,@realsebastiani I post about republicans and d...,3.0,0.0,7.0,0
4,2022-12-30 23:51:47+00:00,@BLKAnonymous @PrettyMill1 @WendellPierce Goog...,1.0,0.0,28.0,0


In [ ]:
df.dtypes

date             object
rawContent       object
replyCount      float64
retweetCount    float64
likeCount       float64
quoteCount       object
dtype: object

#### Converting column datatype

In [ ]:
from numpy import float64
df['quoteCount'] = df['quoteCount'].astype('str')

In [ ]:
# Dropping row 
df = df.drop(df[df['quoteCount'] == "['airtel', 'google']"].index)

In [ ]:
df['quoteCount'] = df['quoteCount'].astype('float64')

In [ ]:
df.dtypes

date             object
rawContent       object
replyCount      float64
retweetCount    float64
likeCount       float64
quoteCount      float64
dtype: object

In [ ]:
pwd

'/content/drive/MyDrive/cloud_computing/Project'

In [ ]:
df.to_csv('Cleaned Data/cleaned_google_tweets.csv',index = False)

#### Reading cleaned csv file into pyspark dataframe

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/cloud_computing/Project/Cleaned Data/cleaned_google_tweets.csv', engine = 'python')

In [ ]:
df.shape

(792010, 6)

In [ ]:
# convert the Pandas DataFrame to a PySpark DataFrame
ggl_twts_df = spark.createDataFrame(df)

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [ ]:
ggl_twts_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|
+--------------------+--------------------+----------+------------+---------+----------+
|2022-12-30 23:59:...|@joshnamesnotjo1 ...|       2.0|         1.0|     97.0|       0.0|
|2022-12-30 23:57:...|"wow, librarians ...|       0.0|         1.0|      6.0|       0.0|
|2022-12-30 23:56:...|i miss being a ki...|       1.0|         1.0|     11.0|       0.0|
|2022-12-30 23:52:...|@realsebastiani I...|       3.0|         0.0|      7.0|       0.0|
|2022-12-30 23:51:...|@BLKAnonymous @Pr...|       1.0|         0.0|     28.0|       0.0|
|2022-12-30 23:51:...|@LauraPowellEsq m...|      12.0|         3.0|    154.0|       0.0|
|2022-12-30 23:50:...|Strategic bombing...|       8.0|         2.0|     54.0|       0.0|
|2022-12-30 23:47:...|Elizabeth Lake, C...|       0.0|         0.0|     12.0|       1.0|
|2022-12-30 23:46:...

### Filtering stock tweets

In [ ]:
import re

def extract_stock_tweets(tweet):
    # Define a regular expression pattern to match stock market-related keywords and hashtags
    pattern = r'\b$googl\b|\bstock market\b|\bstocks\b|\bshares\b|\btrading\b|\binvesting\b|\binvestor\b|\bbullish\b|\bbearish\b|\bportfolio\b|\bETF\b|\bindex\b|\b#stockmarket\b|\bstockmarkets\b|\bstockmarketnews\b|\bstockmarketinvesting\b|\bindianstockmarket\b|\bstockmarketindia\b|\bstockmarketeducation\b|\bstockmarketcrash\b|\bstockmarkettips\b|\bstockmarketquotes\b|\bstockmarkettrader\b|\bpakistanstockmarket\b|\bstockmarketanalysis\b|\bstockmarketlab\b|\bstockmarketinvestor\b|\bstockmarketmemes\b|\bstockmarketca\b|\bstockmarketmindgames\b|\bwoodstockmarket\b|\bstockmarketing\b|\bstockmarketmonitor\b|\bstockmarkettrading\b|\bstockmarketcourse\b|\bstockmarketupdate\b|\busstockmarket\b|\bstockmarket101\b|\bphilippinestockmarket\b|\bstockmarketopportunities\b|\binstastockmarket\b|\bstockmarketprice\b|\bstockmarketph\b|\bthestockmarket\b|\bstockmarketgame\b|\bstockmarketcrash2022\b|\bstockmarketadvisory\b|\bstockmarkettraining\b|\blearnstockmarket\b|\bstockmarketgains\b|\bstockmarketleader\b|\blivestockmarketing\b'

    if re.search(pattern, tweet, re.IGNORECASE):
        return True
    else:
        return False

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType,StringType

# Convert the custom function into a PySpark UDF
custom_filter_udf = udf(extract_stock_tweets, BooleanType())

ggl_stock_twts_df = ggl_twts_df.filter(custom_filter_udf(ggl_twts_df['rawContent']))

In [ ]:
ggl_stock_twts_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|
+--------------------+--------------------+----------+------------+---------+----------+
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|
|2022-12-30 18:52:...|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|
|2022-12-30 17:59:...|This right here! ...|       1.0|         0.0|      8.0|       0.0|
|2022-12-30 17:47:...|There's no questi...|       7.0|         3.0|     47.0|       1.0|
|2022-12-30 16:19:...|Invest in index f...|       0.0|         1.0|     38.0|       0.0|
|2022-12-30 15:34:...|Businesses that c...|       3.0|         6.0|     29.0|       3.0|
|2022-12-30 14:52:...|@avalancheavax @o...|       1.0|         2.0|     14.0|       0.0|
|2022-12-30 14:05:...|A great Google Ad...|       0.0|         0.0|     14.0|       0.0|
|2022-12-30 14:03:...

### Preprocessing the filtered stock tweets

In [ ]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stopwords = stopwords.words('english') 
import nltk
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Define a function to perform lemmatization
def lemmatization(tweet):
    words = tweet.split()
    lem_sentence = " ".join([lemmatizer.lemmatize(word) for word in words])
    return lem_sentence

def remove_stop_words(tweet):
  words = tweet.split()  # split the sentence into words
  filtered_words = [word for word in words if word.casefold() not in stopwords]
  filtered_text = ' '.join(filtered_words)  # join the filtered words into a sentence
  return filtered_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 28.4 MB/s eta 0:00:00


In [ ]:
import re
import contractions

def preprocess_tweets(tweet):
    tweet = tweet.lower() #converting lowercase
    tweet = re.sub(r'@\w+', ' ', tweet) #removes all mentions from the tweet
    tweet = re.sub(r'#\w+', ' ', tweet) #removes all hashtags from the tweet
    tweet = re.sub(r'http\S+', ' ', tweet) #removes all URLs from the tweet
    tweet = re.sub(r'www\S+', ' ', tweet) #removes all URLs that begin with "www" from the tweet
    tweet = re.sub(r'\d+\w*|\w*\d+\w*', ' ', tweet) #replaces alphanumeric or numeric with word number
    tweet = re.sub(r'(.)\1\1+', r'\1\1', tweet) #keep two consecutive duplicates and remove the rest
    tweet = re.sub(r"'s\b", ' ',tweet) # remove 's
    tweet = re.sub(r'[^\w\s]', ' ', tweet) #removes all non-alphanumeric characters and non-whitespace characters from the tweet
    tweet = re.sub(r'[^\x00-\x7F]+', ' ', tweet)  #removes all non-ASCII characters from the tweet 
    tweet = re.sub(r'[^\w\s#@/:%.,_-]', ' ', tweet) #removes any remaining non-alphanumeric characters
    tweet = contractions.fix(tweet)
    tweet = lemmatization(tweet)
    tweet = remove_stop_words(tweet)
    return tweet

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType,StringType

# Convert the preprocessing function into a PySpark UDF
custom_filter_udf = udf(preprocess_tweets, StringType())
ggl_stock_twts_cleaned_df = ggl_stock_twts_df.withColumn("cleanedTweets",custom_filter_udf(ggl_stock_twts_df['rawContent']))

In [ ]:
ggl_stock_twts_cleaned_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+--------------------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|
+--------------------+--------------------+----------+------------+---------+----------+--------------------+
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|
|2022-12-30 18:52:...|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|seen lot people g...|
|2022-12-30 17:59:...|This right here! ...|       1.0|         0.0|      8.0|       0.0|right investing b...|
|2022-12-30 17:47:...|There's no questi...|       7.0|         3.0|     47.0|       1.0|question facebook...|
|2022-12-30 16:19:...|Invest in index f...|       0.0|         1.0|     38.0|       0.0|invest index fund...|
|2022-12-30 15:34:...|Businesses that c...|       3.0|         6.0|     29.0|       3.0|business close fe...|
|2022-12-3

### Calculating sentiment score for tweets

In [ ]:
# Download the Vader lexicon if it is not already downloaded
nltk.download('vader_lexicon')

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a UDF to calculate sentiment score for a text
def calculate_sentiment_score(text):
    # Analyze the sentiment of the text
    scores = analyzer.polarity_scores(text)
    
    # Return the compound sentiment score
    return scores['compound']

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
# Register the UDF
sentiment_udf = udf(calculate_sentiment_score, DoubleType())

ggl_stock_twts_cleaned_df = ggl_stock_twts_cleaned_df.withColumn("sentimentScore",sentiment_udf(ggl_stock_twts_cleaned_df['cleanedTweets']))

In [ ]:
ggl_stock_twts_cleaned_df.show(10)

+--------------------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|                date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|sentimentScore|
+--------------------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|2022-12-30 19:41:...|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|       -0.7269|
|2022-12-30 18:52:...|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|seen lot people g...|       -0.1531|
|2022-12-30 17:59:...|This right here! ...|       1.0|         0.0|      8.0|       0.0|right investing b...|        0.4728|
|2022-12-30 17:47:...|There's no questi...|       7.0|         3.0|     47.0|       1.0|question facebook...|       -0.6705|
|2022-12-30 16:19:...|Invest in index f...|       0.0|         1.0|     38.0|       0.0|invest index fund...|        0.7003|


# Grouping by date

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def extract_date(date):
  date_value = date.split()[0]
  return date_value

# Register the UDF
sentiment_udf = udf(extract_date, StringType())

ggl_stock_twts_cleaned_df = ggl_stock_twts_cleaned_df.withColumn("date",sentiment_udf(ggl_stock_twts_cleaned_df['date']))

In [ ]:
ggl_stock_twts_cleaned_df.show(10)

+----------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|      date|          rawContent|replyCount|retweetCount|likeCount|quoteCount|       cleanedTweets|sentimentScore|
+----------+--------------------+----------+------------+---------+----------+--------------------+--------------+
|2022-12-30|"GLOBAL STOCKS LO...|      11.0|        20.0|     46.0|       0.0|global stock lose...|       -0.7269|
|2022-12-30|I've seen a lot o...|       3.0|         1.0|     16.0|       0.0|seen lot people g...|       -0.1531|
|2022-12-30|This right here! ...|       1.0|         0.0|      8.0|       0.0|right investing b...|        0.4728|
|2022-12-30|There's no questi...|       7.0|         3.0|     47.0|       1.0|question facebook...|       -0.6705|
|2022-12-30|Invest in index f...|       0.0|         1.0|     38.0|       0.0|invest index fund...|        0.7003|
|2022-12-30|Businesses that c...|       3.0|         6.0|     29.0|       3.0|bu

In [ ]:
from pyspark.sql.functions import col, count, sum, avg, expr, percentile_approx, abs

# Group the tweets by date and calculate the daywise positive score, negative score,
# number of positive tweets, and number of negative tweets

grouped_ggl_stock_twts_df = ggl_stock_twts_cleaned_df.groupBy("date").agg(
    (sum(expr("sentimentScore * likeCount")) / sum("likeCount")).alias("weighted_avg_sentiment_score"),
    # Calculate the median positive sentiment score
    percentile_approx(expr("CASE WHEN sentimentScore > 0 THEN sentimentScore END"), 0.5).alias("daywise_median_positive_score"),
    # Calculate the median negative sentiment score
    abs(percentile_approx(expr("CASE WHEN sentimentScore < 0 THEN sentimentScore END"), 0.5)).alias("daywise_median_negative_score"),
    sum((col("sentimentScore") > 0.0).cast("int")).alias("daywise_positive_tweet_count"),
    sum((col("sentimentScore") < 0.0).cast("int")).alias("daywise_negative_tweet_count"),
    avg(col("replyCount")).alias("daywise_avg_replyCount"),
    avg((col("retweetCount")).cast("int")).alias("daywise_avg_retweetCount"),
    avg((col("likeCount")).cast("int")).alias("daywise_avg_likeCount"),
    avg((col("quoteCount")).cast("int")).alias("daywise_avg_quoteCount")
)

In [ ]:
# replacing null values with 0
grouped_ggl_stock_twts_df = grouped_ggl_stock_twts_df.na.fill(value = 0.0)
grouped_ggl_stock_twts_df.show(10)

In [ ]:
grouped_ggl_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+
|2022-01-01|         0.18319239999999998|                       0.3818|                       0.3182|                           9|                           1|     3.769230769230769|       20.23076923076923|    57.69230769230769|    0.923076

In [ ]:
grouped_ggl_stock_twts_df.count()

364

In [ ]:
# Rounding off the values using ceil
from pyspark.sql.functions import ceil

# Assume `df` is a PySpark DataFrame with a column `col` that contains values to be rounded up
grouped_ggl_stock_twts_df = grouped_ggl_stock_twts_df.withColumn("daywise_avg_replyCount", ceil(grouped_ggl_stock_twts_df.daywise_avg_replyCount))
grouped_ggl_stock_twts_df = grouped_ggl_stock_twts_df.withColumn("daywise_avg_retweetCount", ceil(grouped_ggl_stock_twts_df.daywise_avg_retweetCount))
grouped_ggl_stock_twts_df = grouped_ggl_stock_twts_df.withColumn("daywise_avg_likeCount", ceil(grouped_ggl_stock_twts_df.daywise_avg_likeCount))
grouped_ggl_stock_twts_df = grouped_ggl_stock_twts_df.withColumn("daywise_avg_quoteCount", ceil(grouped_ggl_stock_twts_df.daywise_avg_quoteCount))


In [ ]:
company_id = {'google':1,'microsoft':2, 'tesla':3, 'amazon':4, 'apple':5}

In [ ]:
from pyspark.sql.functions import lit
grouped_ggl_stock_twts_df = grouped_ggl_stock_twts_df.withColumn('company',lit(company_id['google']))

In [ ]:
grouped_ggl_stock_twts_df.show(10)

+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+-------+
|      date|weighted_avg_sentiment_score|daywise_median_positive_score|daywise_median_negative_score|daywise_positive_tweet_count|daywise_negative_tweet_count|daywise_avg_replyCount|daywise_avg_retweetCount|daywise_avg_likeCount|daywise_avg_quoteCount|company|
+----------+----------------------------+-----------------------------+-----------------------------+----------------------------+----------------------------+----------------------+------------------------+---------------------+----------------------+-------+
|2022-01-01|         0.18319239999999998|                       0.3818|                       0.3182|                           9|                           1|                     4|                      21|          

## Saving the file

In [ ]:
# converting pyspark dataframe to pandas dataframe
DF = grouped_ggl_stock_twts_df.toPandas()

DF.to_csv('Preprocessed Data/google_preprocessed.csv', index = False)

**Note**: Similarly preprocessing is done for other companies like AMAZON, GOOGLE, TESLA, MICROSOFT, APPLE